In [75]:
import io
from google.oauth2 import service_account
from google.cloud import speech
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
import time

In [2]:
client_file = 'data/keys/google_sa.json'
credentials = service_account.Credentials.from_service_account_file(client_file)

In [3]:
DATA_DIR = Path('data/')
OUTPUT_DIR = Path('output/')

## STT

In [22]:
audio_file = 'data/datasets/audio/4.wav'
with io.open(audio_file, 'rb') as f:
    content = f.read()
audio = speech.RecognitionAudio(content=content)

In [23]:
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=24000,
    language_code='en-US'
)

In [24]:
response = client.recognize(config=config, audio=audio)

In [25]:
response

results {
  alternatives {
    transcript: "enable value transformation for status"
    confidence: 0.955208957
  }
  result_end_time {
    seconds: 2
    nanos: 850000000
  }
  language_code: "en-us"
}
results {
  alternatives {
    transcript: " percent x + 2 with unit km per hour for garage input 3"
    confidence: 0.865534902
  }
  result_end_time {
    seconds: 9
    nanos: 370000000
  }
  language_code: "en-us"
}
total_billed_time {
  seconds: 10
}
request_id: 3567256788358102650

In [26]:
text = ''
for res in response.results:
    text += res.alternatives[0].transcript
text

'enable value transformation for status percent x + 2 with unit km per hour for garage input 3'

In [18]:
print(response.results[0].alternatives[0].transcript)

what's the temperature in Sydney


### Pipeline

In [66]:
output_df = pd.DataFrame(columns=['id', 'text'])

client = speech.SpeechClient(credentials=credentials)

for i, p in enumerate(sorted(list((DATA_DIR / f'datasets/audio').glob('*')), key=lambda x: int(x.stem))):
    with io.open(p, 'rb') as f:
        content = f.read()
    audio = speech.RecognitionAudio(content=content)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=24000,
        language_code='en-US'
    )

    try:
        response = client.recognize(config=config, audio=audio)
    except Exception as ex:
        print(ex)
        time.sleep(15)
        response = client.recognize(config=config, audio=audio)
        
    text = ''
    for res in response.results:
        text += res.alternatives[0].transcript

    output_df.loc[len(output_df)] = pd.Series({'id': p.stem, 'text': text})

    if i == 0:
        header = True
        mode = 'w'
    else:
        header = False
        mode = 'a'
    output_df.iloc[[len(output_df)-1]].to_csv(DATA_DIR / f'datasets/stt.csv', index=False, header=header, mode=mode)

## TTS

In [27]:
import google.cloud.texttospeech as tts

In [32]:
def text_to_wav(voice_name: str, text: str, output_path):
    language_code = "-".join(voice_name.split("-")[:2])
    text_input = tts.SynthesisInput(text=text)
    voice_params = tts.VoiceSelectionParams(
        language_code=language_code, name=voice_name
    )
    audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16)

    client = tts.TextToSpeechClient(credentials=credentials)
    response = client.synthesize_speech(
        input=text_input,
        voice=voice_params,
        audio_config=audio_config,
    )

    with open(output_path, "wb") as out:
        out.write(response.audio_content)
    
    return response

In [10]:
text_to_wav("en-US-Journey-D", "What is the temperature in Sydney?")

Generated speech saved to "data/audio/en-US-Journey-D.wav"


In [40]:
response = text_to_wav("en-US-Journey-F", ("Reset all Living room cover 1 counters.").lower(), "test.wav")

In [41]:
("Reset all Living room cover 1 counters.").lower()

'reset all living room cover 1 counters.'

In [42]:
response

audio_content: "RIFF$+\002\000WAVEfmt \020\000\000\000\001\000\001\000\300]\000\000\200\273\000\000\002\000\020\000data\000+\002\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\001\000\000\000\001\000\000\000\000\000\000\000\001\000\002\000\003\000\002\000\002\000\001\000\001\000\002\000\001\000\003\000\002\000\003\000\003\000\010\000\006\000\002\000\004\000\004\000\005\000\006\000\006\000\005\000\005\000\007\000\007\000\007\000\010\000\007\000\010\000\t\000\n\000\n\000\t\000\013\000\013\000\013\000\014\000\014\000\013\000\013\000\014\000\r\000\r\000\r\000\016\000\017\000\016\000\017\000\017\000\016\000\017\000\016\000\017\000\016\000\r\000\021\000\020\000\020\000\017\000\020\000\020\000\021\000\021\000\020\000\021\000\021\000\021\000\021\000\021\000\022\000\020\000\021\000\021\000\020\000\020\000\017\000\021\000\020\000\020\000\020\000\021\000\021\000\020\000\020\000\017

### Pipeline

In [61]:
df = pd.read_csv(DATA_DIR / 'datasets/tts.csv')

In [ ]:
import time

for i, row in df.iloc[118:].iterrows():
    print(i)
    
    user_cmd = row['user_cmd'].lower()
    output_path = DATA_DIR / f'datasets/audio/{row["id"]}.wav'

    if i < len(df) / 2:
        voice_name = "en-US-Journey-F"
    else:
        voice_name = "en-US-Journey-D"
    
    try:
        text_to_wav(voice_name, user_cmd, output_path)
    except Exception as ex:
        print(ex)
        time.sleep(15)
        text_to_wav(voice_name, user_cmd, output_path)

## Evaluation

In [67]:
from openai import OpenAI
with open('data/keys/openai.txt') as f:
    key = f.read()
client = OpenAI(api_key=key)

In [68]:
stt_df = pd.read_csv(OUTPUT_DIR / 'stt/output.csv')
tts_df = pd.read_csv(DATA_DIR / 'datasets/tts.csv')
res_df = pd.DataFrame(columns=['id', 'reference', 'recognized', 'same'])

for i, r in stt_df.iterrows():
    print(i)
    
    id = r['id']
    rec = r['text']
    ref = tts_df[tts_df['id'] == id].iloc[0]['user_cmd']

    with open(DATA_DIR / 'prompts/stt/validation.md') as f:
        instruction = f.read()
    
    prompt = (f'{instruction}\n\n'
              f'Sentence 1: {ref}\n'
              f'Sentence 2: {rec}')

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    if completion.choices[0].message.content == 'True':
        same = 1
    elif completion.choices[0].message.content == 'False':
        same = 0
    else:
        same = -1

    res_df.loc[len(res_df)] = pd.Series({'id': id, 'reference': ref, 'recognized': rec, 'same': same})

    if i == stt_df.index[0]:
        header = True
        mode = 'w'
    else:
        header = False
        mode = 'a'
    res_df.iloc[[len(res_df)-1]].to_csv(OUTPUT_DIR / 'stt/compared.csv', index=False, header=header, mode=mode)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [69]:
df = pd.read_csv(OUTPUT_DIR / 'stt/compared.csv')

In [70]:
df[df['same'] == -1].shape[0]

0

In [71]:
df[df['same'] == 1].shape[0] / df.shape[0]

0.5109717868338558

In [76]:
display(df[df['same'] == 0])

,id,reference,recognized,same
1,1,Get the active energy consumption data for the Outdoor Light 4.,get the active energy consumption data for the outdoor light for,0
5,5,"Check the configuration settings of the Home office Input 2, please.",check the configuration settings of the home office and put two please,0
6,6,Retrieve the settings of the Bedroom Switch 4.,retrieve the settings of the bedroom switch for,0
9,9,Turn on the Bedroom Light 4 at 50% brightness with a transition of 2 seconds.,turn on the bedroom light for at 50% brightness with a transition of two seconds,0
10,10,Set the temperature for Bathroom Temperature 2 to 22°C.,set the temperature for bath room temperature to 22 22 degrees C,0
13,13,Please provide the voltage limit for the Bedroom Light 2.,please provide the voltage limit for the bedroom light to,0
14,14,Reset all counters of the Living room Input 4.,reset all counters of the living room input for,0
15,15,"Hey, can you tell me the current temperature with Greenhouse Temperature 4?",hey can you tell me the current temperature with Greenhouse temperature for,0
16,16,Reset all available counters for this Basement Input 4.,reset all available counters for the spaceman input for,0
17,17,"Is there any Garage Humidity 4 offset set for the room? If yes, can you provide it?",is there any garage vanity for offset set for the room if yes can you provide it,0
